In [64]:
def mse(v1, v2):
    return ((np.array(v1) - np.array(v2))**2).mean()

In [95]:
class Regressor(object):
    def __init__(self, lista_pesos, alpha,lambda_,x,y):
        # Inicialitzem w0 i w1 (per ser ampliat amb altres w's)
        self.lista_pesos = lista_pesos
        self.lambda_=lambda_
        self.alpha = alpha
        self.x = x
        self.y=y

        
    def predict(self, x):
        return(sum([self.lista_pesos[i+1]*x[i] for i in range(len(x))])+self.lista_pesos[0])
            
    
    def __update(self, hy, y,x):
        self.lista_pesos[0]=self.lista_pesos[0]-self.alpha*(1/(len(y)))*(sum([hy[i]-y[i] for i in range (len(y))]))
        for i in range(len(self.lista_pesos)-1):
            self.lista_pesos[i+1]=self.lista_pesos[i+1]- self.alpha*((1/(len(y)))*((sum(([(hy[j]-y[j])*x[j][i]  for j in range (len(y))]))))-(self.lambda_/len(y))*self.lista_pesos[i+1])
            
        # actualitzar aqui els pesos donada la prediccio (hy) i la y real.
        
    
    def train(self, max_iter, epsilon):
        # Entrenar durant max_iter iteracions o fins que la millora sigui inferior a epsilon
        for j in range(max_iter):
            lista= (self.lista_pesos).copy()
            ypred=[self.predict(self.x[i]) for i in range(len(self.x)) ]
            self.__update(ypred,self.y,self.x)
            if(mse(self.lista_pesos,lista)<epsilon ):
                print(mse(self.lista_pesos,lista))
                print("epsilooon")
                print(j)
                break
               
        print("iteeeer")

In [66]:
from sklearn.datasets import make_regression
import numpy as np
import pandas as pd
#matplotlib notebook
from matplotlib import pyplot as plt
import scipy.stats
import math 

# Visualitzarem només 3 decimals per mostra
pd.set_option('display.float_format', lambda x: '%.3f' % x)

# Funcio per a llegir dades en format csv
def load_dataset(path):
    dataset = pd.read_csv(path, header=0, delimiter=',',decimal=',')
    return dataset

# Carreguem dataset d'exemple
dataset = load_dataset('measurements.csv')
data = dataset.values
dataset.describe()
x = dataset.drop(columns='consume')
y = data[:, 1]
print("Dimensionalitat de la BBDD:", dataset.shape)
print("Dimensionalitat de les entrades X", x.shape)
print("Dimensionalitat de l'atribut Y", y.shape)

print("Per comptar el nombre de valors no existents:")
print(dataset.isnull().sum())

del(dataset['refill liters'])
del(dataset['refill gas'])
del(dataset['specials'])

dataset.loc[dataset['gas_type'] == 'E10', "gas_type"] = 1
dataset.loc[dataset['gas_type'] == 'SP98', "gas_type"] = 0

dataset['temp_inside']=dataset['temp_inside'].fillna(dataset["temp_inside"].mean())
dataset['distance']=1/(dataset['distance'])
#print(dataset['distance'])
dataset['speed']=1/(dataset['speed'])
dataset['time'] = dataset['distance']/dataset['speed']
print(min(dataset['time']))
x = dataset.drop(columns='consume')
x = x.drop(columns='temp_inside')
x = x.drop(columns='speed')
x = x.drop(columns='sun')
x = x.drop(columns='gas_type')
x = x.drop(columns='time')
x=x.values
y = dataset.filter(['consume'])
y=y.values


Dimensionalitat de la BBDD: (388, 12)
Dimensionalitat de les entrades X (388, 11)
Dimensionalitat de l'atribut Y (388,)
Per comptar el nombre de valors no existents:
distance           0
consume            0
speed              0
temp_inside       12
temp_outside       0
specials         295
gas_type           0
AC                 0
rain               0
sun                0
refill liters    375
refill gas       375
dtype: int64
0.37914691943127965


In [93]:
def split_data(x, y, train_ratio=0.8):
    np.random.seed(5)
    indices = np.arange(x.shape[0])
    np.random.shuffle(indices)
    n_train = int(np.floor(x.shape[0]*train_ratio))
    indices_train = indices[:n_train]
    indices_val = indices[n_train:] 
    x_train = x[indices_train, :]
    y_train = y[indices_train]
    x_val = x[indices_val, :]
    y_val = y[indices_val]
    return x_train, y_train, x_val, y_val

# Dividim dades d'entrenament


In [135]:
x_train, y_train, x_val, y_val = split_data(x, y)
prueba=Regressor([5,8,0,1,1], 0.01,0.6,x_train,y_train)
prueba.train(100000, 0.0000001)

9.964394239545255e-08
epsilooon
384
iteeeer


In [136]:
prueba.lista_pesos

[array([4.6695952]),
 array([8.02698696]),
 array([-0.04848573]),
 array([0.85831468]),
 array([0.74491467])]

In [137]:
from sklearn.metrics import r2_score
r2 = r2_score(y_val,[prueba.predict(x_val[i]) for i in range(len(x_val)) ])
print("R2 score en atribut : %f" %(r2))

R2 score en atribut : 0.646013
